In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

from google.colab import drive
drive.mount("/content/gdrive")

!mkdir -p drive
!google-drive-ocamlfuse drive

!fusermount -u drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.26-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [2]:
!unzip "/content/gdrive/My Drive/spectrograms_split1.zip" -d "/content"

Streaming output truncated to the last 5000 lines.
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00044_3.wav_augmented.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00044_4.wav.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00044_4.wav_augmented.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00044_5.wav.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00044_5.wav_augmented.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00045_1.wav.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00045_1.wav_augmented.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00045_2.wav.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00045_3.wav.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00045_3.wav_augmented.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00045_4.wav.png  
  inflating: /conte

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import glob
import tensorflow.keras
from tensorflow import keras
from sklearn import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten


In [4]:

!pip install -q keras
!pip install keras-metrics

In [5]:
import tensorflow.keras.metrics
batch_size=64
number_of_class=10
img_rows,img_clns=360,360


In [6]:
IMG_SIZE = 360

train_images = []
y_train = []

for directory_path in glob.glob("/content/spectrograms_split1/train/*"):
  label = directory_path.split("/")[-1]

  for img_path in glob.glob(os.path.join(directory_path, "*.png")):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    train_images.append(img)
    y_train.append(label)

y_train = np.array(y_train)
train_images = np.array(train_images)

y_test = []
test_images = []

for directory_path in glob.glob("/content/spectrograms_split1/test/*"):
  label = directory_path.split("/")[-1]
  for img_path in glob.glob(os.path.join(directory_path, "*.png")):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    test_images.append(img)
    y_test.append(label)

y_test = np.array(y_test)
test_images = np.array(test_images)


In [8]:
le = preprocessing.LabelEncoder()
le.fit(y_test)
y_test = le.transform(y_test)
le.fit(y_train)
y_train = le.transform(y_train)


y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir='/content/spectrograms_split1/train'
test_dir='/content/spectrograms_split1/test'


test_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_rows,img_clns),  # all images will be resizes to this size
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical')

test_steps = test_generator.n // batch_size

train_datagen = ImageDataGenerator(rescale=1.0/255.0)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_rows,img_clns),  # all images will be resizes to this size
        batch_size=batch_size,
        shuffle=True,
        class_mode='categorical')
train_steps = train_generator.n // batch_size

Found 1000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.


In [11]:
model = tensorflow.keras.Sequential()

model.add(tensorflow.keras.layers.Conv2D(128, input_shape=(img_rows,img_clns,3), kernel_size=3, strides=4, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=tensorflow.keras.regularizers.l2(l=0.0001)))
model.add(tensorflow.keras.layers.BatchNormalization())
model.add(tensorflow.keras.layers.Activation('relu'))
model.add(tensorflow.keras.layers.MaxPooling2D(pool_size=3, strides=None))

model.add(tensorflow.keras.layers.Conv2D(128, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=tensorflow.keras.regularizers.l2(l=0.0001)))
model.add(tensorflow.keras.layers.BatchNormalization())
model.add(tensorflow.keras.layers.Activation('relu'))
model.add(tensorflow.keras.layers.MaxPooling2D(pool_size=3, strides=None))

model.add(tensorflow.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=tensorflow.keras.regularizers.l2(l=0.0001)))
model.add(tensorflow.keras.layers.BatchNormalization())
model.add(tensorflow.keras.layers.Activation('relu'))
model.add(tensorflow.keras.layers.MaxPooling2D(pool_size=3, strides=None))

model.add(tensorflow.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=tensorflow.keras.regularizers.l2(l=0.0001)))
model.add(tensorflow.keras.layers.BatchNormalization())
model.add(tensorflow.keras.layers.Activation('relu'))
model.add(tensorflow.keras.layers.MaxPooling2D(pool_size=3, strides=None,name='feature_layer'))




model.add(tensorflow.keras.layers.Lambda(lambda x : tensorflow.keras.backend.mean(x, axis=1)))
model.add(tensorflow.keras.layers.Flatten())
model.add(tensorflow.keras.layers.Dense(10, activation='softmax'))

opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=[['accuracy'],tensorflow.keras.metrics.Recall(),tensorflow.keras.metrics.Precision()])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 90, 90, 128)       3584      
_________________________________________________________________
batch_normalization_4 (Batch (None, 90, 90, 128)       512       
_________________________________________________________________
activation_4 (Activation)    (None, 90, 90, 128)       0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 128)       147584    
_________________________________________________________________
batch_normalization_5 (Batch (None, 30, 30, 128)       512       
_________________________________________________________________
activation_5 (Activation)    (None, 30, 30, 128)      

In [12]:
history = model.fit(train_images, y_train_one_hot, steps_per_epoch= train_steps+1, epochs=20, validation_data=(test_images, y_test_one_hot),validation_steps=1)

Epoch 1/20
141/141 [==============================] - 22s 44ms/step - loss: 1.5415 - accuracy: 0.4704 - recall_1: 0.2879 - precision_1: 0.6707 - val_loss: 5.8562 - val_accuracy: 0.1610 - val_recall_1: 0.1540 - val_precision_1: 0.1623
Epoch 2/20
141/141 [==============================] - 5s 36ms/step - loss: 1.1145 - accuracy: 0.6259 - recall_1: 0.4929 - precision_1: 0.7402 - val_loss: 2.1648 - val_accuracy: 0.4180 - val_recall_1: 0.3850 - val_precision_1: 0.4923
Epoch 3/20
141/141 [==============================] - 5s 36ms/step - loss: 0.9706 - accuracy: 0.6752 - recall_1: 0.5800 - precision_1: 0.7663 - val_loss: 2.0665 - val_accuracy: 0.4890 - val_recall_1: 0.4460 - val_precision_1: 0.5386
Epoch 4/20
141/141 [==============================] - 5s 36ms/step - loss: 0.8299 - accuracy: 0.7278 - recall_1: 0.6512 - precision_1: 0.8055 - val_loss: 2.2990 - val_accuracy: 0.3540 - val_recall_1: 0.3250 - val_precision_1: 0.3727
Epoch 5/20
141/141 [==============================] - 5s 36ms/step 

In [13]:
feature_model = Model(inputs=model.input, outputs=model.get_layer('feature_layer').output)

X_feature_train = feature_model.predict(train_images)
X_feature_train = X_feature_train.reshape(X_feature_train.shape[0],-1)


X_feature_test = feature_model.predict(test_images)
X_feature_test = X_feature_test.reshape(X_feature_test.shape[0],-1)


In [16]:
ml_models=[neural_network.MLPClassifier(max_iter=2000, activation='tanh',solver='lbfgs'),
           linear_model.LogisticRegression(dual=False,multi_class='auto',solver='lbfgs'),
           ensemble.RandomForestClassifier(n_estimators=100,criterion='entropy'),
           discriminant_analysis.LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto'),
           neighbors.KNeighborsClassifier(n_neighbors=1, weights='distance'),
           svm.SVC(kernel='rbf',C=100, gamma='auto'),
           naive_bayes.GaussianNB(),
           ensemble.GradientBoostingClassifier(),
           ensemble.AdaBoostClassifier(),
           svm.LinearSVC(penalty='l1',dual=False,multi_class='crammer_singer',max_iter=1000000)]



for ml_model in ml_models:
  ml_model.fit(X_feature_train, y_train)
  prediction = ml_model.predict(X_feature_test)
  accuracy = metrics.accuracy_score(y_test, prediction)
  print(ml_model)
  print(accuracy)

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=2000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)
0.828


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
0.848


KeyboardInterrupt: ignored